# Extreme Spring Boot
This notebook contains my notes from the *Extreme Spring Boot* talk by Craig Walls at UberConf 2019. I expected this talk to be a sort of "Advanced Spring Boot" class, but it was really more like "Beginning Spring Boot Part 2". As a result, I didn't hear a ton of new things but did get some useful tidbits.

Much of the session was live-coding. It can be found at https://github.com/habuma/nfjs-samples in the *uberconf-2019* folder.

### Improving Startup Time
It isn't unusual to see 20-30 seconds startup time for reasonably sized apps.

Spring does use reflection. Over time the Spring team has been trying to reduce usages of reflection, but there are still places where Spring uses reflection.

Craig says startup time only matters in a few places:

* Serverless functions (Lambda, etc.)
* Integration tests
* Development time

While I agree with him that those are the most important, I would personally like fast startup everywhere. It can be tricky to have a load balancer health checking your app when it takes so long to start up.

Some of Craig's suggestions to improve startup time:

* Add the `spring-context-indexer` JAR on large projects
    * The default behavior of component scanning the classpath at startup-time is slow.
    * The indexer does component scanning at build-time instead of startup-time.
    * This works well for big projects, but can actually be slower for really small projects (i.e. test it to see whether it works for yours)
* Explicitly any actuators you don't need
    * Do you need to do this for the endpoint and MBean? I think you do.
* Upgrade to the latest version of Spring
    * 2.1 improved startup time quite a bit over 2.0.
    * Craig said 2.2 should improve startup time more.
* Explicit `spring.config.location` 
    * Boot by default will look in multiple places for config. You can use this to configure exactly where to look for config
* spring.jmx.enabled = false
    * How does turning this off impact AppDynamics? I don't know if it relies on JMX for some things.
* Make beans lazy by default
    * Can use @Lazy on individual beans
    * In 2.2 you'll be able to set a property that defaults all beans to lazy.
* Functional bean definition (Ja-Fu)
    * You register beans in code instead of using component scanning or other standard forms of config.
    * This helps avoid things like having to do annotation processing
    * This can have very big performance gains, but looks pretty ugly
* GraalVM native image
    * This is very bleeding-edge but has potential for huge improvements.

Using Ja-Fu and GraalVM native image combination will net huge performance results, but is likely a huge pain to get set up at the moment, particularly the GraalVM piece.

### Using Spring DevTools
Spring Boot has a spring-boot-devtools JAR that will help you at development time. To use it, just add `spring-boot-devtools` as a dependency to your project.

I believe this works only when you use the Spring Boot plugin (`spring-boot:run`) to run your application. 

You don't need to worry about having this dependency in your POM file go to production with your application. When building the actual JAR file, the spring-boot-plugin won't ever add that JAR. Additionally, Spring Boot auto-configuration won't enable devtools when running inside a JAR or WAR file, even if devtools is available.

Devtools will automatically restart your application when source files change. The restart only reloads some classes in the JVM, so it is much faster than the initial cold start.

### Properties
Using `@ConfigurationProperties(prefix="myprefix")` to add groupings of properties to a class.  This is handy for groupings of properties that are used in multiple places.

### Actuator
Future session will discuss this more in-depth.

All *management* properties that you can configure in Spring Boot are for the actuator. 

##### /env endpoint
This endpoint shows information about the configured Spring Environment. You can use this to inspect all the PropertySources that have been configured. I just wrote some code to do this the other day while debugging and didn't realize it was in the actuator! 

### ASCII Art Banner
You can use an image for the banner and it will convert to ASCII art!!! Sometimes the resulting art looks great, other times it looks terrible.

The file has to be called *banner.fileExtension*, such as *banner.jpg*. And yes, it works with animated GIFs too.

### Takeaways
I would like to try the following:

* Attempt to use some of the performance suggestions to improve our startup time. Understand the tradeoffs that come with making those improvements.
* Figure out how to get devtools working with an application when it is inside a Docker container. I would like to use docker-compose to easily wire up my app to dependencies, so it would be nice to be able to do that while still using devtools.
* Get the FS logo and put it in as the ASCII art banner to see how it looks.